In [1]:
import tensorflow as tf
import numpy as np

C:\Users\pwend\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
bandits=[0.2,0,-0.2,-5]
num_bandits=len(bandits)
def pullBandit(bandit):
    result=np.random.randn(1)
    if result>bandit:
        return 1
    else:
        return -1

In [6]:
tf.reset_default_graph()
weights=tf.Variable(tf.ones([num_bandits]))
chosen_action=tf.argmax(weights,0)
reward_holder=tf.placeholder(shape=[1],dtype=tf.float32)
action_holder=tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight=tf.slice(weights,action_holder,[1])
loss=-(tf.log(responsible_weight)*reward_holder)
optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.001)
update=optimizer.minimize(loss)

In [10]:
total_episodes=1000
total_rewards=np.zeros(num_bandits)
total_reward=np.zeros(num_bandits)
e=0.1
init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    i=0
    while i<total_episodes:
        if np.random.randn(1)<e:
            action=np.random.randint(num_bandits)
        else:
            action=sess.run(chosen_action)
        
        reward=pullBandit(bandits[action])
        
        _,resp,ww=sess.run([update,responsible_weight,weights],feed_dict={reward_holder:[reward],action_holder:[action]})
        
        total_reward[action]+=reward
        if i % 50 == 0:
            print("Running reward for the " + str(num_bandits) + " bandits: " + str(total_reward))
        i+=1
print("The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print("...and it was right!")
else:
    print("...and it was wrong!")

Running reward for the 4 bandits: [-1.  0.  0.  0.]
Running reward for the 4 bandits: [-1. -2.  1. 25.]
Running reward for the 4 bandits: [-8.  0.  2. 55.]
Running reward for the 4 bandits: [-7.  0.  0. 84.]
Running reward for the 4 bandits: [ -8.  -2.   3. 118.]
Running reward for the 4 bandits: [ -7.  -5.   1. 144.]
Running reward for the 4 bandits: [-10.  -1.   2. 172.]
Running reward for the 4 bandits: [-14.  -7.   2. 202.]
Running reward for the 4 bandits: [-15.  -3.   5. 234.]
Running reward for the 4 bandits: [-14.  -3.   9. 265.]
Running reward for the 4 bandits: [-11.  -2.  11. 297.]
Running reward for the 4 bandits: [-15.  -2.  12. 334.]
Running reward for the 4 bandits: [-17.   2.  10. 366.]
Running reward for the 4 bandits: [-16.   3.  10. 394.]
Running reward for the 4 bandits: [-18.   4.  10. 421.]
Running reward for the 4 bandits: [-15.   6.   4. 450.]
Running reward for the 4 bandits: [-10.   5.   2. 474.]
Running reward for the 4 bandits: [-10.   5.   5. 501.]
Running 